## Import Modules

In [2]:

import pandas as pd

## Request page source from URL

In [3]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [4]:
from selenium import webdriver
import time
import pandas as pd
import os

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC


In [5]:
!pip install webdriver-manager

Defaulting to user installation because normal site-packages is not writeable


In [20]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=service)
url='https://www.linkedin.com/jobs/search/?currentJobId=3997507725&geoId=106967730&keywords=Marketing%20Data%20Analyst&location=Berlin%2C%20Berlin%2C%20Germany'
driver.get(url)
driver.implicitly_wait(10)




In [21]:

WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, 'base-search-card'))
)

[<selenium.webdriver.remote.webelement.WebElement (session="fa16e3f8d9a2799339af7d259df3271b", element="f.0BD8E683EE9816B50153D41F0088EC83.d.9FBCFA117C173F83DF87F46C3C81BE7B.e.102")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fa16e3f8d9a2799339af7d259df3271b", element="f.0BD8E683EE9816B50153D41F0088EC83.d.9FBCFA117C173F83DF87F46C3C81BE7B.e.103")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fa16e3f8d9a2799339af7d259df3271b", element="f.0BD8E683EE9816B50153D41F0088EC83.d.9FBCFA117C173F83DF87F46C3C81BE7B.e.104")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fa16e3f8d9a2799339af7d259df3271b", element="f.0BD8E683EE9816B50153D41F0088EC83.d.9FBCFA117C173F83DF87F46C3C81BE7B.e.105")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fa16e3f8d9a2799339af7d259df3271b", element="f.0BD8E683EE9816B50153D41F0088EC83.d.9FBCFA117C173F83DF87F46C3C81BE7B.e.106")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fa16e3f8d9a2799339af

In [24]:
companyname=[]
titlename=[]
location=[]

In [34]:

n = driver.find_elements(By.CLASS_NAME, 'base-search-card__subtitle')
try:
    for i in range(len(n)):
        company=driver.find_elements(By.CLASS_NAME,'base-search-card__subtitle' )[i].text
        title=driver.find_elements(By.CLASS_NAME,'base-search-card__title' )[i].text
        loc=driver.find_elements(By.CLASS_NAME,'job-search-card__location' )[i].text
        companyname.append(company)
        titlename.append(title)
        location.append(loc)
    
except IndexError:
    print('no')



In [35]:
print(len(companyname),len(titlename),len(location))

140 140 140


In [36]:
titlefinal=pd.DataFrame(titlename,columns=['Title'])
companyfinal=pd.DataFrame(companyname,columns=['Company'])
locfinal=pd.DataFrame(location,columns=['Location'])

In [37]:

final_df = pd.concat([titlefinal, companyfinal, locfinal], axis=1)
final_df


,Title,Company,Location
0,Data Analyst (all genders),Sunday Natural,"Berlin, Berlin, Germany"
1,"Data Analyst / Decision Scientist, Digital Mar...",Vinted,"Berlin, Berlin, Germany"
2,Analytical Consultant (Fixed-Term Contract),Google,"Berlin, Berlin, Germany"
3,"Business Analyst (m/w/d), befristet bis 31.12....",Coca-Cola Europacific Partners,"Berlin, Berlin, Germany"
4,Junior Social Media Analyst (w/m/d),TLGG Agency,"Berlin, Berlin, Germany"
...,...,...,...
135,Business Analyst (w/m/d) - Sales Operations,smava,"Berlin, Germany"
136,AMS Europe Business Analyst L2 Banking Core,Avaloq,"Berlin, Berlin, Germany"
137,Business Analyst / IT Requirements Engineer (a...,PŸUR | Tele Columbus,"Berlin, Berlin, Germany"
138,Business Analyst (m/f/d),Inkitt,"Berlin, Berlin, Germany"


In [38]:
final_df.isnull().sum()

Title       0
Company     0
Location    0
dtype: int64

In [39]:
final_df.columns

Index(['Title', 'Company', 'Location'], dtype='object')

In [40]:
final_df.to_csv('LinkedIn_scrape_data.csv',index=False)

## Import Data into SQL

In [16]:
!pip install pymysql

Defaulting to user installation because normal site-packages is not writeable


In [17]:
import pymysql

In [41]:
connection=pymysql.connect(
    host='localhost',
    user='root',
    password='p1234',
    database='da_jobs_new'
)

create_table_query='''
create table if not exists job_listings(
id INT AUTO_INCREMENT PRIMARY KEY,
Title varchar(255),
Company varchar(255),
Location varchar(255)
);
'''

cursor=connection.cursor()
cursor.execute(create_table_query)

insert_query = '''
    INSERT INTO job_listings (Title, Company, Location) VALUES (%s, %s, %s);
'''

for index, job in final_df.iterrows():
    cursor.execute(insert_query, (job['Title'], job['Company'], job['Location']))  # Combined skills


connection.commit()
cursor.close()